In [25]:
import numpy as np
import pandas as pd

In [26]:
pd.set_option('display.max_columns', 500)

---

In [27]:
loan = pd.read_csv('../../data/raw/Loan.txt', sep='\t')
borrower = pd.read_csv('../../data/raw/Borrower.txt', sep='\t')

In [28]:
data_loan_borrowers = pd.merge(loan, borrower, on='memberId')

---

In [29]:
data_loan_borrowers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   loanId                    100000 non-null  int64  
 1   memberId                  100000 non-null  int64  
 2   date                      100000 non-null  object 
 3   purpose                   100000 non-null  object 
 4   isJointApplication        99029 non-null   float64
 5   loanAmount                98994 non-null   float64
 6   term                      98929 non-null   object 
 7   interestRate              100000 non-null  float64
 8   monthlyPayment            100000 non-null  int64  
 9   grade                     100000 non-null  object 
 10  loanStatus                100000 non-null  object 
 11  residentialState          100000 non-null  object 
 12  yearsEmployment           100000 non-null  object 
 13  homeOwnership             100000 non-null  ob

---

In [30]:
loan_borrowers = data_loan_borrowers.copy()

**Fix column names**

In [31]:
column_names = ['loan_id', 'member_id', 'date', 'purpose', 'is_joint_application', 'loan_amount', 'term', 'interest_rate', 'monthly_payment', 'grade', 'loan_status', 'residential_state', 'years_employment', 'home_ownership', 'annual_income', 'income_verified', 'dti_ratio', 'length_credit_history', 'n_total_credit_lines', 'n_open_credit_lines', 'n_open_credit_lines_1_year', 'revolving_balance', 'revolving_utilization_rate', 'n_derogatory_record', 'n_delinquency_2_years', 'n_charge_off_1_year', 'n_inquiries_6_months']

In [32]:
loan_borrowers.columns = column_names

**Convert date to datetime format**

In [33]:
loan_borrowers['date'] = loan_borrowers['date'].apply(pd.to_datetime)

**Extract the term number**

In [34]:
loan_borrowers['term_in_months'] = data_loan_borrowers['term'].str.split(' ').str[0].apply(lambda x: int(x) if not pd.isnull(x) else x) 

**Separate purpose levels with space**

In [35]:
data_loan_borrowers.purpose.unique()

array(['debtconsolidation', 'homeimprovement', 'healthcare', 'education',
       'other', 'business', 'auto'], dtype=object)

In [36]:
purpose_space = {'debtconsolidation': 'Debt Consolidation', 'homeimprovement': 'Home Improvement'}
loan_borrowers['purpose'] = loan_borrowers['purpose'].apply(lambda x: purpose_space[x] if x in purpose_space else x.capitalize())

**Impute missing loan amount/terms by calculating it using monthly payment, interest rate, and terms/loan amount**

In [37]:
loan_info = ['loan_amount', 'term_in_months', 'interest_rate', 'monthly_payment']
loan_borrowers_without_amount_or_term = loan_borrowers.loc[loan_borrowers['loan_amount'].isna() ^ loan_borrowers['term_in_months'].isna()]
loan_borrowers_no_amount = loan_borrowers.loc[loan_borrowers['loan_amount'].isna() & ~loan_borrowers['term_in_months'].isna()]
loan_borrowers_no_term = loan_borrowers.loc[~loan_borrowers['loan_amount'].isna() & loan_borrowers['term_in_months'].isna()]

In [38]:
loan_borrowers_no_amount.shape[0] + loan_borrowers_no_term.shape[0] == loan_borrowers_without_amount_or_term.shape[0]

True

In [39]:
from typing import Optional
def calculate_loan_info(row: pd.Series, info: str) -> Optional[int]:
   """Calculate the missing loan amount or number of terms given its number
   of terms in months or loan amount, respectively, yearly interest rate, and 
   amortized monthly payment.

   - info == 'amount' / 'terms'
   """
   emi = row['monthly_payment']  # amortized equal monthly installment
   r = row['interest_rate'] / 1200  # monthly interest rate

   if info == 'amount':
      n = row['term_in_months']  # number of terms 
      principal = emi * (1 - (1/(1+r)) ** n) / r  # total loan amount
      return np.round(principal)
   elif info == 'terms':
      p = row['loan_amount']  # total loan amount
      numerator = emi / (emi - p * r)
      terms_in_months = np.log(numerator) / np.log(1 + r)  # number of terms
      return np.round(terms_in_months)
   else:
      print('info has to be either "amount" or "terms". Try again.')

In [40]:
loan_borrowers.loc[loan_borrowers_no_amount.index, 'loan_amount'] = loan_borrowers_no_amount[loan_info].apply(lambda row: calculate_loan_info(row, info='amount'), axis=1)

In [41]:
loan_borrowers.loc[loan_borrowers_no_term.index, 'term_in_months'] = loan_borrowers_no_term[loan_info].apply(lambda row: calculate_loan_info(row, info='terms'), axis=1)

In [42]:
loan_borrowers.loc[loan_borrowers_without_amount_or_term.index, ['loan_amount', 'term_in_months']].isna().sum()

loan_amount       0
term_in_months    0
dtype: int64

Great! All records where either loan amount or term in months are missing are now filled.

In [43]:
loan_borrowers.term_in_months.value_counts()

60.0    34318
48.0    32869
36.0    32796
Name: term_in_months, dtype: int64

**Fill in the term column the additionally calculated terms**

In [44]:
loan_borrowers.loc[loan_borrowers_no_term.index, 'term'] = loan_borrowers.loc[loan_borrowers_no_term.index, 'term_in_months'].apply(lambda x: f'{int(x)} months')

**Verify the columns loan amount, term, interest rate, and monthly payment using the equal monthly installment formula**

In [45]:
borrowers_with_full_loan_detail = loan_borrowers[~loan_borrowers['loan_amount'].isna() | ~loan_borrowers['loan_amount'].isna()]

In [46]:
emi = borrowers_with_full_loan_detail['monthly_payment']
p = borrowers_with_full_loan_detail['loan_amount']
r = borrowers_with_full_loan_detail['interest_rate'] / 1200
n = borrowers_with_full_loan_detail['term_in_months']

In [52]:
np.sum(emi == np.round(p * r * (1 + r) ** n / ((1 + r) ** n - 1))) == borrowers_with_full_loan_detail.shape[0]

True

---

# <center>**Load the transformed data to the data/interim folder for EDA**</center> 

In [48]:
loan_borrowers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   loan_id                     100000 non-null  int64         
 1   member_id                   100000 non-null  int64         
 2   date                        100000 non-null  datetime64[ns]
 3   purpose                     100000 non-null  object        
 4   is_joint_application        99029 non-null   float64       
 5   loan_amount                 99983 non-null   float64       
 6   term                        99983 non-null   object        
 7   interest_rate               100000 non-null  float64       
 8   monthly_payment             100000 non-null  int64         
 9   grade                       100000 non-null  object        
 10  loan_status                 100000 non-null  object        
 11  residential_state           100000 non-n

**Reorder the columns to put term_in_months behind term**

In [49]:
cols = loan_borrowers.columns.to_list()
cols = cols[:7] + [cols[-1]] + cols[7:-1]

In [50]:
loan_borrowers = loan_borrowers[cols]

In [54]:
loan_borrowers.to_csv('../../data/interim/loan_borrower.csv', index=False, mode='w+')